## 7단계: 역전파 자동화

> 역전파 코드를 직접 작성하면 실수가 생길 수 있고, 무엇보다 지루할 것입니다. \
그래서 이제부터 역전파를 자동화하려 합니다. \
더 정확히 말하면, 일반적인 계산(순전파)을 한 번만 해주면 어떤 계산이라도 상관없이 역전파가 자동으로 이루어지는 구조를 만들 것입니다. \
두둥! 지금부터가 바로 Define-by-Run의 핵심을 건드리는 내용입니다!

### 7.1 역전파 자동화의 시작

우선 역전파 자동화를 위해 변수와 함수의 '관계'를 이해해보자.

함수 입장에서 변수는 '입력'과 '출력'에 쓰인다.

변수는 함수에 의해 '만들어진다'. (창조자인 함수가 존재하지 않으면 사용자에 의해 만들어졌다 간주하자)

이러한 관계는 아래 그림으로 나타낼 수 있다. (점선은 참조(reference)를 뜻함)

<img src="images/그림 7-2.png" width=600/>

이를 코드에 녹여보자.

In [1]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)  # 출력 변수에 창조자 설정
        self.input = input
        self.output = output  # 출력도 저장한다. -> '연결'을 동적으로 만드는 기법의 핵심
        return output

In [2]:
# 필요 모듈 정의

import numpy as np

class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [ ]:
# 이와 같이 '연결'된 Variable과 Function이 있다면
# 계산 그래프를 거꾸로 거슬러 올라갈 수 있다.
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 계산 그래프의 노드들을 거꾸로 거슬러 올라간다.
assert y.creator == C  # assert 문은 True가 아니면 AssertionError 발생
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

위의 관계는 다음 그림으로 나타낼 수 있다.

<img src="images/그림 7-3.png" width=600/>

중요한 점은 위 계산 그래프는 실제로 계산을 수행하는 시점(순전파 시점)에 만들어진다는 것이다.

이러한 특성을 Define-by-Run이라 한다.

또한 위 계산 그래프처럼 노드들의 연결로 이루어진 데이터 구조를 '링크드 리스트(linked list)'라고 한다.

### 7.2 역전파 도전!

y에서 b까지의 역전파 구현

<img src="images/그림 p77_01.png" width=600/>

In [5]:
y.grad = np.array(1.0)

C = y.creator  # 1. 함수를 가져온다.
b = C.input  # 2. 함수의 입력을 가져온다.
b.grad = C.backward(y.grad)  # 3. 함수의 backward 메서드를 호출한다.

b에서 a까지의 역전파 구현

<img src="images/그림 p77_02.png" width=600/>

In [6]:
B = b.creator  # 1. 함수를 가져온다.
a = B.input  # 2. 함수의 입력을 가져온다.
a.grad = B.backward(b.grad)  # 3. 함수의 backward 메서드를 호출한다.

a에서 x까지의 역전파 구현

<img src="images/그림 p77_03.png" width=600/>

In [8]:
A = a.creator  # 1. 함수를 가져온다.
x = A.input  # 2. 함수의 입력을 가져온다.
x.grad = A.backward(a.grad)  # 3. 함수의 backward 메서드를 호출한다.

# 역전파 구현 완료
print(x.grad)

3.297442541400256


### 7.3 backward 메서드 추가

방금의 역전파 코드는 똑같은 처리가 반복되었다.

이를 자동화할 수 있도록 Variable 클래스에 backward 메서드를 추가한다.

In [9]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        f = self.creator  # 1. 함수를 가져온다.
        if f is not None:
            x = f.input  # 2. 함수의 입력을 가져온다.
            x.grad = f.backward(self.grad)  # 3. 함수의 backward 메서드를 호출한다.
            x.backward()  # 하나 앞 변수의 backward 메서드를 호출한다. (재귀)

Variable의 backward 메서드는 지금까지의 처리 흐름과 거의 동일하다.
1. Variable.creator에서 함수 가져옴
2. 함수의 입력 변수 가져옴
3. 함수의 backward 메서드를 호출함
4. 하나 앞 변수의 backward 메서드를 호출함

이런 식으로 각 변수의 backward 메서드가 재귀적으로 호출됨.

(Variable.creator가 None이면 역전파가 중단됨)

이제 자동화된 역전파를 확인해보자.

In [10]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256
